In [ ]:
!pip install pyspark
!pip install datetime

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import MapType,StringType
from pyspark.sql.functions import from_unixtime, unix_timestamp, date_format
import pandas as pd
from datetime import datetime

In [ ]:
df = pd.read_excel('/content/inconsistent_data.xlsx')

In [ ]:
print(df.dtypes)

S#                           int64
Date                        object
Islamic Date                object
Blast Day Type              object
Holiday Type                object
Time                        object
City                        object
Latitude                   float64
Longitude                   object
Province                    object
Location                    object
Location Category           object
Location Sensitivity        object
Open/Closed Space           object
Influencing Event/Event     object
Target Type                 object
Targeted Sect if any        object
Killed Min                 float64
Killed Max                 float64
Injured Min                float64
Injured Max                 object
No. of Suicide Blasts      float64
Explosive Weight (max)      object
Hospital Names              object
Temperature(C)             float64
Temperature(F)             float64
dtype: object


In [ ]:
df.shape

(496, 26)

In [ ]:
df.isnull().sum()

S#                           0
Date                         0
Islamic Date               154
Blast Day Type              10
Holiday Type               424
Time                       213
City                         0
Latitude                     3
Longitude                    3
Province                     0
Location                     3
Location Category           36
Location Sensitivity        36
Open/Closed Space           35
Influencing Event/Event    306
Target Type                 26
Targeted Sect if any        48
Killed Min                 146
Killed Max                  16
Injured Min                131
Injured Max                 32
No. of Suicide Blasts       82
Explosive Weight (max)     324
Hospital Names             199
Temperature(C)               5
Temperature(F)               7
dtype: int64

In [ ]:
df2 = df.rename(columns={"S#": "S/N"})


In [ ]:
df2 = df2.drop_duplicates()

In [ ]:
df2.head()

,S/N,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,Sunday-November 19-1995,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,NaN,Islamabad,33.7180,73.0718,Capital,...,None,14.0,15.0,NaN,60,2.0,NaN,NaN,15.835,60.503
1,2,Monday-November 6-2000,10 SHa`baan 1421 A.H,Working Day,NaN,NaN,Karachi,24.9918,66.9911,Sindh,...,None,NaN,3.0,NaN,3,1.0,NaN,NaN,23.770,74.786
2,3,Wednesday-May 8-2002,25 safar 1423 A.H,Working Day,NaN,7:45 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,13.0,15.0,20.0,40,1.0,2.5 Kg,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628
3,4,Friday-June 14-2002,3 Raby` al-THaany 1423 A.H,Working Day,NaN,11:10:00 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,NaN,12.0,NaN,51,1.0,NaN,NaN,31.430,88.574
4,5,Friday-July 4-2003,4 Jumaada al-awal 1424 A.H,Working Day,NaN,NaN,Quetta,30.2095,67.0182,Baluchistan,...,Shiite,44.0,47.0,NaN,65,1.0,NaN,1.CMH Quetta \n2.Civil Hospital 3. Boland Medi...,33.120,91.616


In [48]:
df2['Killed Min'].fillna(0, inplace= True)
df2['Injured Min'].fillna(0, inplace= True)
df2['No. of Suicide Blasts'].fillna(1, inplace= True)
df2['Explosive Weight (max)'].fillna(0, inplace= True)

In [ ]:
df2['Killed Max'] = df2['Killed Max'].fillna(method='bfill', axis=0)

In [ ]:
df2['Killed Min'] = df2['Killed Min'].astype(int)
df2['Killed Max'] = df2['Killed Max'].astype(int)
df2['Injured Min'] = df2['Injured Min'].astype(int)
df2['No. of Suicide Blasts'] = df2['No. of Suicide Blasts'].astype(int)


In [ ]:
def parse_date(s):
    # Remove everything before the first "-"
    s = s.split("-", 1)[1]

    # Try different date formats
    formats = ["%b %d-%Y", "%B %d-%Y", "%b-%d-%Y", "%B-%d-%Y"]
    for fmt in formats:
        try:
            return datetime.strptime(s, fmt).strftime('%Y-%m-%d')
        except ValueError:
            continue
    raise ValueError(f"Date {s} doesn't match known formats")

df2['Date'] = df2['Date'].apply(parse_date)

In [ ]:
df2['Killed Max']

0      15
1       3
2      15
3      12
4      47
       ..
491    22
492     7
493     2
494     3
495     4
Name: Killed Max, Length: 496, dtype: int64

In [ ]:
df2['Islamic Date'] = df2['Islamic Date'].str.lower()

In [38]:
df2['Explosive Weight (max)']= df2['Explosive Weight (max)'].str.rstrip('kg')
df2['Explosive Weight (max)']= df2['Explosive Weight (max)'].str.rstrip('KG')
df2['Explosive Weight (max)']= df2['Explosive Weight (max)'].str.rstrip('kG')
df2['Explosive Weight (max)']= df2['Explosive Weight (max)'].str.rstrip('Kg')

In [50]:
df2['Temperature(C)'] = df2['Temperature(C)'].round(2)
df2['Temperature(F)'] = df2['Temperature(F)'].round(2)

In [49]:
df2.head(10)


,S/N,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,1995-11-19,25 jumaada al-thaany 1416 a.h,Holiday,Weekend,NaN,Islamabad,33.7180,73.0718,Capital,...,None,14,15,0,60,2,0.0,NaN,15.835,60.503
1,2,2000-11-06,10 sha`baan 1421 a.h,Working Day,NaN,NaN,Karachi,24.9918,66.9911,Sindh,...,None,0,3,0,3,1,0.0,NaN,23.770,74.786
2,3,2002-05-08,25 safar 1423 a.h,Working Day,NaN,7:45 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,13,15,20,40,1,2.5,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628
3,4,2002-06-14,3 raby` al-thaany 1423 a.h,Working Day,NaN,11:10:00 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,0,12,0,51,1,0.0,NaN,31.430,88.574
4,5,2003-07-04,4 jumaada al-awal 1424 a.h,Working Day,NaN,NaN,Quetta,30.2095,67.0182,Baluchistan,...,Shiite,44,47,0,65,1,0.0,1.CMH Quetta \n2.Civil Hospital 3. Boland Medi...,33.120,91.616
5,6,2003-12-25,2 thw al-qi`dah 1424 a.h.,Holiday,Christmas/birthday of Quaid-e-Azam,1:40:00 PM/1:42:00 PM,Rawalpindi,33.6058,73.0437,Punjab,...,None,16,18,0,50,2,30.0,1.District headquarters \nHospital,9.445,49.001
6,7,2004-02-28,8 muharram 1425 a.h,Working Day,NaN,NaN,Rawalpindi,33.6058,73.0437,Punjab,...,shiite,0,1,3,4,1,0.0,NaN,15.275,59.495
7,8,2004-05-07,18 raby` al-awal 1425 a.h,Working Day,NaN,1:20:00/1:17:00 PM,Karachi,24.9918,66.9911,Sindh,...,shiite,14,15,96,200,1,2.0,Civil hospital,33.860,92.948
8,9,2004-05-31,NaN,Working Day,NaN,evening,Karachi,24.9918,66.9911,Sindh,...,Shiite,16,18,0,35,1,0.0,Civil hospital-Liaquat National hospital-Jinna...,31.605,88.889
9,10,2004-06-03,15 raby` al-thaany 1425 a.h,Working Day,NaN,NaN,North waziristan,32.9746,70.1456,FATA,...,None,0,2,0,2,1,0.0,NaN,27.180,80.924
